<a href="https://colab.research.google.com/github/Nachi2006/MLREPO/blob/main/NeuralNetworktoCML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# This script demonstrates how to create a PyTorch model and convert it to a Core ML .mlpackage file.

# First, make sure you have the necessary libraries installed.
# pip install torch coremltools scikit-learn numpy

import torch
import torch.nn as nn
import coremltools as ct
import warnings
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import SGD
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Suppress warnings that might appear during conversion.
warnings.filterwarnings("ignore", category=UserWarning)

# Step 1: Define a simple PyTorch model for the Iris dataset.
# The Iris dataset has 4 features and 3 classes.
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)  # Input layer with 4 features, hidden layer with 16 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)   # Hidden layer to output layer with 3 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model and set it to evaluation mode.
# This is crucial for conversion as it disables things like dropout layers.
model = IrisNet()
model.eval()

# Step 2: Load and preprocess the Iris dataset.
print("Loading and preparing the Iris dataset...")
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert numpy arrays to PyTorch tensors.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
optimizer = SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Step 3: Train the model for a few epochs.
print("Training the model...")
for epoch in range(50):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print("Training complete.")
print("-" * 30)

# Step 4: Create a dummy input tensor for tracing.
# The shape must match the expected input of the model. Here, it's a single sample with 4 features.
dummy_input = torch.tensor(X_test[0:1], dtype=torch.float32)

# Step 5: Trace the model using torch.jit.trace.
# This step creates a TorchScript representation of the model's graph.
traced_model = torch.jit.trace(model, dummy_input)

# Step 6: Convert the traced PyTorch model to Core ML.
# We use the ct.convert() function and specify the input type.
# The `inputs` parameter is a list of ct.TensorType or ct.ImageType.
coreml_model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=dummy_input.shape)]
)

# Step 7: Save the Core ML model to a file.
# Newer versions of coremltools default to the "mlprogram" format, which uses the .mlpackage extension.
# We will use .mlpackage as it is the recommended and modern format.
coreml_model.save("iris_net.mlpackage")

print("PyTorch model successfully converted and saved as iris_net.mlpackage")
print("-" * 30)

# The verification step has been removed as `coreml_model.predict()` is not supported in this environment.
# To verify the conversion, you would need to load the generated .mlpackage file on a macOS system
# or an iOS/iPadOS device and run a prediction with it.

Loading and preparing the Iris dataset...
Training the model...


Training complete.
------------------------------


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 3695.96 passes/s]


PyTorch model successfully converted and saved as iris_net.mlpackage
------------------------------
